In [ ]:
from __main__ import *
 
def sigmoid(x, derivative=False):
    sigm = 1. / (1. + np.exp(-x))
    if derivative:
        return sigm * (1. - sigm)
    return sigm
 
 
def gradient_single(X, y, w, lmbda):
    # compute gradient with respect to a single sample and assign to output
 
    return output
 
 
def f_gradient(X, y, w):
    yX = y[:, np.newaxis] * X
    l1 = np.matmul(yX, w)
    return -sigmoid(-l1)[:, np.newaxis] * yX
 
 
def reg_gradient(w, lmbda):
    return 2 * lmbda * w
 
 
def gradient(X, y, w, lmbda):
    return reg_gradient(w, lmbda) + np.mean(f_gradient(X, y, w), axis=0)
 
 
def gradient_full(X, y, w, lmbda):
    # Compute the full gradient of the given function 
    
    return output
 
 
def cost(X, y, w, lmbda):
     # Define the cost function
 
    return output
 
def binary_classification_cost(X, y, w):
    return f1_score(y_true = y, y_pred = np.sign(np.matmul(X, w)), average='micro')
 
def ax_modifier(ax, legend_loc, ncol, xlabel, ylabel, title=None):
    ymin, ymax = ax.get_ylim()
    xmin, xmax = ax.get_xlim()
    no_curves = len(ax.lines)
    ymin = min(ymin, min([min(ax.lines[i].get_ydata()) for i in range(no_curves)]))
    ymax = max(ymax, max([max(ax.lines[i].get_ydata()) for i in range(no_curves)]))
    xmax = max(xmax, max([max(ax.lines[i].get_xdata()) for i in range(no_curves)]))
    ax.legend(loc=legend_loc)
    ax.set_xlabel(xlabel, fontsize=16)
    ax.set_ylabel(ylabel, fontsize=16)
    ax.set_xticks(np.arange(0, xmax+1, step=round(xmax/15)))
    ax.legend(ncol=ncol)
    if title is not None:
        ax.set_title(title)
    if not(np.isinf([ymin, ymax]).all()):  #if boundaries are defined
        ax.set_ylim((0.98*ymin, 1.02*ymax))
 
 
def random_sampler(N, batch=1, buffersize=10000):
    """
    A generator of random indices from 0 to N.
    params:
    N: upper bound of the indices
    batch: Number of indices to return per iteration
    buffersize: Number of numbers to generate per batch
                (this is only a computational nicety)
    """
 
    S = int(np.ceil(buffersize / batch))
 
    while True:
        buffer = np.random.randint(N, size=(S, batch))
        for i in range(S):
            yield buffer[i]
 
 
def GD(X, y, w, learning_rate=0.1, lmbda=0.01, iterations=1000):
    for iteration in range(iterations):
        # update the parameter w
 
        yield w
 
 
def SGD(X, y, w, lmbda, learning_rate, batch_size=1):
    N, D = X.shape
    sampler = random_sampler(N, batch_size)
 
    for ix in sampler:
       # update the parameter w 
    
        yield w
 
 
def SVRG(X, y, w, lmbda, learning_rate, epoch_size):
    """
    Stochastic variance reduced gradient
    """
 
    sampler = random_sampler(X.shape[0], epoch_size)
 
    for epoch in sampler:
        full_grad = gradient_full(X, y, w, lmbda)
        mean_grad = np.mean(full_grad, axis=0)
 
        # write inner loop of SVRG using full gradient vectors computed in full_grad and its mean value mean_grad
    
 
        yield w
 
 
def SAG(X, y, w, lmbda, learning_rate, batch_size=1):
    N, D = X.shape
    P, = w.shape
    sampler = random_sampler(N, batch_size)
 
    grad = np.zeros((N, P))
    delta = np.zeros(P)
    non_zero_v = np.zeros(N)
    m = 0
    for ix in sampler:
        # update the number of seen examples m
        m -= np.sum(non_zero_v[ix], axis=0)
        non_zero_v[ix] = 1
        m += np.sum(non_zero_v[ix], axis=0)
 
        # complete the update the sum of the gradient
      
         # update the parameter w 
        yield w
 
 
def initialize_w(N):
    return np.random.randn(N)
 
 
def loss(X, y, w, lmbda):
    objective_loss = cost(X, y, w, lmbda)
    # compute f1_score
    
    return objective_loss, f1_score
 
 
def iterate(opt, X_train, y_train, X_test, y_test, w_0,
            lmbda, iterations=100, inner=1, name="NoName", printout=True):
    """
    This function takes an optimizer and returns a loss history for the
    training and test sets.
    """
 
    loss_hist_train, train_f1_score = loss(X_train, y_train, w_0, lmbda)
    loss_hist_test, test_f1_score = loss(X_test, y_test, w_0, lmbda)
    mean_grad_norm = np.linalg.norm(np.mean(gradient_full(X_train, y_train, w_0, lmbda), axis=0), 2)
 
    ws = [w_0]
    clock = [0]
 
    start = time.time()
    for iteration in range(iterations):
        #print("\n ------Iteration {} is started-----".format(iteration))
        for _ in range(inner):
            w = next(opt)
        clock.append(time.time() - start)
        ws.append(w)
 
    #for iteration, w in enumerate(ws):
        train_loss, train_f1_score_new = loss(X_train, y_train, w, lmbda)
        loss_hist_train = np.append(loss_hist_train, train_loss)
        train_f1_score = np.append(train_f1_score, train_f1_score_new)
 
        test_loss, test_f1_score_new = loss(X_test, y_test, w, lmbda)
        loss_hist_test = np.append(loss_hist_test, test_loss)
        test_f1_score = np.append(test_f1_score, test_f1_score_new)
        grad_norm_new = np.linalg.norm(np.mean(gradient_full(X_train, y_train, w, lmbda), axis=0), 2)
        mean_grad_norm = np.append(mean_grad_norm, grad_norm_new)
 
        if printout:
            print('{}; Iter = {:02}; Objective(train) = {:05.3f}; Objective(test) = {:05.3f}; F1score(train) = {:05.3f}; F1score(test) = {:05.3f}'.format(name, iteration, train_loss, test_loss, train_f1_score_new, test_f1_score_new))
        #print('Current solution = {}'.format(w.round(decimals=3)))
        sys.stdout.flush()
 
    return ws[-1], loss_hist_train, loss_hist_test, train_f1_score, test_f1_score, clock, mean_grad_norm
 
 
def optimizer_binary_classification(target_dataset, X_train, y_train,
                                                    X_test, y_test, w_0,
                                                    hyper_parameters, multi_class=False, printout=True):
    optimizers = [
         {
                    "opt": GD(X=X_train, y=y_train, w=w_0, lmbda=hyper_parameters['lambda'],
                              learning_rate=hyper_parameters['learning_rate'],
                              iterations = hyper_parameters['iterations']),
                    "name": "GD",
                    "inner": 1
            },
            {
                # complete SGD
                   # "opt": SGD() 
            },
            {
                # complete SAG
                  #  "opt": SAG()
            },
            {
                # complete SVRG
                  #  "opt": SVRG()
            },
           
    ]
 
    outputs = {optimizers[i]['name']: {'optimal_parameter': {}, 'training_loss': {}, 'training_f1_score': {}, 'training_mean_grad_norm': {}}
               for i in np.arange(len(optimizers))}
 
    fig, ax = plt.subplots(3, 1, figsize=(13, 12))
 
    for opt in optimizers:
 
        w, loss_hist_train, loss_hist_test, train_f1_score, test_f1_score, clock, mean_grad_norm = iterate(
                opt['opt'],
                X_train, y_train, X_test, y_test, w_0,
                lmbda=hyper_parameters['lambda'],
                iterations=hyper_parameters['iterations'], inner=opt['inner'],
                name=opt['name'], printout=printout)
        outputs[opt['name']]['optimal_parameter'] = w
        outputs[opt['name']]['training_loss'] = loss_hist_train
        outputs[opt['name']]['training_f1_score'] = train_f1_score
        outputs[opt['name']]['training_mean_grad_norm'] = mean_grad_norm
 
        color = next(ax[0]._get_lines.prop_cycler)['color']
        iterations_axis = range(0, hyper_parameters['iterations'] + 1)
        ax[0].plot(iterations_axis, loss_hist_train,
               label="Train loss ({})".format(opt['name']), linestyle="-", color=color)
 
        ax[0].plot(iterations_axis, loss_hist_test,
               label="Test loss ({})".format(opt['name']), linestyle="--", color=color)
 
        ax[1].plot(iterations_axis, mean_grad_norm,
               label="Training ({})".format(opt['name']), linestyle="-", color=color)
 
        #ax[2].plot(iterations_axis, train_f1_score,
        #       label="Train F1 score ({})".format(opt['name']), linestyle="-", color=color)
 
        ax[2].plot(iterations_axis, test_f1_score,
               label="{}".format(opt['name']), linestyle="-", color=color)
 
        #ax[3].plot(clock, loss_hist_train,
        #           label="Train loss ({})".format(opt['name']), linestyle="-", color=color)
        #ax[3].plot(clock, loss_hist_test,
        #           label="Test loss ({})".format(opt['name']), linestyle="--", color=color)
 
    ax_modifier(ax=ax[0], legend_loc="upper right", ncol=2, xlabel="Iteration", ylabel="Loss",
                title="Performance Comparison of various algorithms")
    ax_modifier(ax=ax[1], legend_loc="upper right", ncol=2, xlabel="Iteration", ylabel="Gradient norm (training)")
    ax_modifier(ax=ax[2], legend_loc="lower right", ncol=2, xlabel="Iteration", ylabel="F1 score (test)")
    #ax_modifier(ax=ax[3], legend_loc="upper right", xlabel="Time in seconds", ylabel="Loss")
 
    if multi_class:
        fig_name = './TestResults/noQuant_allAlg_'+target_dataset+'_Class'+str(y_train[0])
    else:
        fig_name = './TestResults/noQuant_allAlg_'+target_dataset+'_BinaryClassification'
 
    plt.savefig(fig_name+'.png')
    plt.savefig(fig_name+'.pdf')
    matplotlib2tikz.save(fig_name+'.tex')
 
    return outputs